# SVM Linear

In [1]:
# aplicar normalização dos dados

## Importações

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## Leitura de Dados

In [3]:
treinamento = pd.read_csv('diabetes_train.csv')

In [4]:
teste = pd.read_csv('diabetes_test.csv')

## Pré-processamento

In [5]:
class Encoders:
    def __init__(self):
        self.encoders = {}

    def add_encoder(self, feature, encoder):
        self.encoders[feature] = encoder

    def get_encoder(self, feature):
        try:
            return self.encoders[feature]
        except KeyError:
            return None

In [6]:
encoders = Encoders()

In [7]:
def cria_encoder(feature, nome, encoders):
    encoder = LabelEncoder()
    feature_codificada = encoder.fit_transform(feature)
    encoders.add_encoder(nome, encoder)
    return feature_codificada


In [8]:
def verifica_features_categoricas(dados, encoders):
    for i in range(int(dados.size/len(dados)) - 1):
        feature = dados.iloc[:, i]
        if not all(isinstance(instancia, (int, float)) for instancia in (feature.values).tolist()):
            y = encoders.get_encoder(feature.name)
            if y == None:
                dados.loc[:, feature.name] = cria_encoder((feature.values).tolist(), feature.name, encoders)
            else:
                dados.loc[:, feature.name] = y.transform((feature.values).tolist())
    
    return dados

In [9]:
treinamento = verifica_features_categoricas(treinamento, encoders)
teste = verifica_features_categoricas(teste, encoders)

In [10]:
x_treinamento, x_teste = treinamento.iloc[:, :-1].values, teste.iloc[:, :-1].values

In [11]:
y_treinamento, y_teste = treinamento.iloc[:, -1].values, teste.iloc[:, -1].values

## Algoritmo

In [12]:
class SVMLinear:
    def __init__(self, alpha = 1, b = 0, c = 1, tolerancia = 0.0001):
        self.taxa_aprendizado = alpha
        self.param_b = b
        self.param_c = c
        self.tolerancia = tolerancia
        self.max_iteracoes = 1000
        self.qtd_features = None
        self.qtd_instancias = None
        self.parametros = None
        self.qtd_iteracoes = 0

    def fit(self, x, y):
        self.qtd_instancias = len(x)
        self.qtd_features = int(x.size/self.qtd_instancias)
        self.parametros = np.ones(self.qtd_features).reshape(-1, 1)

        y = np.where(y == 0, -1, 1).reshape(-1, 1)

        erro = 1000

        while(erro > self.tolerancia and self.qtd_iteracoes < self.max_iteracoes):
            modelo_svm = np.dot(x, self.parametros) + self.param_b
            classificacao_confianca = y * modelo_svm
            mascara_gradiente = np.where(classificacao_confianca < 1, 1, 0)
            vetor_gradiente_parametros = (self.parametros + self.param_c * np.dot(x.T, (-y * mascara_gradiente)))/self.qtd_instancias
            vetor_gradiente_b = self.param_c * np.sum(-y * mascara_gradiente) / self.qtd_instancias
            novos_parametros = self.parametros - (vetor_gradiente_parametros * self.taxa_aprendizado)
            self.param_b -= (vetor_gradiente_b) * (self.taxa_aprendizado)
            erro = np.mean(abs(self.parametros - novos_parametros))
            self.parametros = novos_parametros
            self.qtd_iteracoes += 1

    def predict(self, x):
        modelo_svm = (np.dot(x, self.parametros)) + self.param_b
        return np.where(modelo_svm < 0, 0, 1).flatten()

In [13]:
svm = SVMLinear()
svm.fit(x_treinamento, y_treinamento)
y_resultado = svm.predict(x_teste)

## Métricas

In [14]:
acuracia = np.sum((np.array(y_teste) == np.array(y_resultado)).astype(int))/len(y_teste) * 100

In [15]:
print(f"Acurácia: {acuracia:.2f}%")

Acurácia: 90.38%


In [16]:
matriz_confusao = np.zeros((2, 2))

for i, k in zip(y_teste, y_resultado):
    matriz_confusao[i][k] += 1

In [17]:
matriz_confusao

array([[ 73.,   8.],
       [ 12., 115.]])